# Similarity measures # 

* ** Cosine similarity** : This computes a unit vector for each row in a matrix. The dot product of each against every other is computed, which gives us the cosine angle between each vector. The end result is a single metric which tells us how close two images are. This is preferred in NLP tasks.
* ** Chi-squared kernel** :  This returns a scalar value that is a measure of closeness. This is preferred in image based tasks.

** Bag of visual words **

To better understand images, we use a cluster of pixels instead of looking at each pixel individually. It is similar to bag of words in the sense that it ignores the spatial order of its visual vocabulary. 

** Transfer learning **

We use deep NN and train them for a specific task. We then cut off the final layers of the network in order to use the lower levels as our features for a task they were not specifically trained on. For example: in the digit recognition exercise, the lower layers may be about representing loopy-ness or straightness while the higher levels may be about 0 or 1-ness. 

In [2]:
# Renewed license until Nov 2nd, 2017.
import graphlab
graphlab.canvas.set_target('ipynb')

## Step 1: Load the dataset ##

We will be using the CIFAR-10 dataset from http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data.

Our objective here is to find the image that is most similar to my own.

In [3]:
gl_img = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data')
gl_img

This non-commercial license of GraphLab Create for academic use is assigned to adarsh108@gmail.com and will expire on November 03, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1478061123.log


Downloading http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data/dir_archive.ini to /var/tmp/graphlab-adarshnair/2078/06515a2c-90a0-4f1b-b507-294193e88029.ini

Downloading http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data/objects.bin to /var/tmp/graphlab-adarshnair/2078/a8b2ee26-a26a-498a-aa6f-56e8456db0f0.bin

Downloading http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data/m_8b0619884fada761.frame_idx to /var/tmp/graphlab-adarshnair/2078/e26f161c-9474-4398-b023-32cdc05c915a.frame_idx

Downloading http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data/m_8b0619884fada761.sidx to /var/tmp/graphlab-adarshnair/2078/564bb480-58da-45bb-8319-7b20a86d3d3b.sidx

Downloading http://s3.amazonaws.com/dato-datasets/coursera/deep_learning/image_train_data/m_8b0619884fada761.0000 to /var/tmp/graphlab-adarshnair/2078/1a722c00-c49e-48d0-a5fe-b5d8551a2382.0000

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


** Deep features ** are features that have been extracted from a large trained deep learning network. 

In [4]:
'''
Check an image
'''
gl_img['image'][0:5].show()

In [5]:
'''
Resize images
'''
graphlab.image_analysis.resize(gl_img['image'][2:3], 96,96).show()

## Step 2: Load your image ##

In [7]:
img = graphlab.Image('pic.png')

# Load image into SArray
ppsf = graphlab.SArray([img])

# Resize image
ppsf = graphlab.image_analysis.resize(ppsf, 32,32)
ppsf.show()

In [8]:
'''
Extract features from image
'''
ppsf = graphlab.SFrame(ppsf).rename({'X1': 'image'})
ppsf

image
Height: 32 Width: 32


In [9]:
'''
Extract deep learning features
'''

'\nExtract deep learning features\n'